# Spark assignment 1: Pairs

Find all the pairs of two consequent words where the first word is “narodnaya”. For each pair, count the number of occurrences in the Wikipedia dump. Print all the pairs with their count in a lexicographical order. Output format is “word_pair <tab> count”, for example:

    red_apple 100500
    crazy_zoo 42

Note that two words in a pair are concatenated with the underscore character, and the result is in the lowercase.

One motivation for counting these continuations is to get a better understanding of the language. Some words, like “the”, have a lot of continuations, while others, like “San”, have just a few (“San Francisco”, for example). One can build a language model with these statistics. If you are interested to learn more, search for “n-gram language model” in the Internet.

The result on the sample dataset:

    narodnaya_gazeta    1
    narodnaya_volya 9
    
If you want to deploy the environment on your own machine, please use [bigdatateam/spark-course1](https://hub.docker.com/r/bigdatateam/spark-course1/) Docker container.

In [1]:
#! /usr/bin/env python

from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local[2]"))

import re

def parse_article(line):
    try:
        article_id, text = line.rstrip().split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []
    
def pairs(words):
    out = []
    for w1, w2 in zip(words, words[1:]):
        out.append((w1.lower() + "_" + w2.lower(), 1))
    return out
    
result = (sc.textFile("/data/wiki/en_articles_part/articles-part", 16)
        .map(parse_article)
        .flatMap(pairs)
        .reduceByKey(lambda x,y : x+y)
        .filter(lambda value: value[0][:9] == "narodnaya")
       ).collect()

for key, count in result:
    print("%s\t%d" % (key, count))

narodnaya_gazeta 1
narodnaya_volya 9
